In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize
import zipfile
import pandas as pd
from tqdm.auto import tqdm
from numpy import expand_dims
import numpy as np

In [ ]:
with zipfile.ZipFile("/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip","r") as z:
    z.extractall(".")

In [ ]:
def append_ext(fn):
    return fn + ".jpg"

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
testdf = pd.read_csv('../input/galaxy-zoo-the-galaxy-challenge/all_zeros_benchmark.zip')
testdf["id"] = testdf['GalaxyID'].astype(str).apply(append_ext)

In [ ]:
from tensorflow.keras.models import load_model
model_path = '../input/test-model/my_h5_model_1.h5'
model = load_model(model_path, custom_objects={'root_mean_squared_error': root_mean_squared_error})

In [ ]:
classes = ['Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1','Class3.2', 
           'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3','Class5.4', 'Class6.1', 
           'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3','Class8.1', 'Class8.2', 'Class8.3', 
           'Class8.4', 'Class8.5', 'Class8.6', 'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 
           'Class10.1', 'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 
           'Class11.4', 'Class11.5', 'Class11.6']

ycols = ['GalaxyID'] + classes

def crop_image(image):
  #Image - numpy array of rank 3
  #Crop image to half its size, preserving the center
  #resize image to shape
  im_size_x = image.shape[0]
  im_size_y = image.shape[0]
  cropx = im_size_x // 4
  cropy = im_size_y // 4
  image = image[cropx:im_size_x-cropx, cropy:im_size_y-cropy]

  re_shape = [64,64]
  image = resize(image, re_shape)
  return image

## Import test data with ImageDataGenerator
testdatagen = ImageDataGenerator(
    fill_mode='constant',
    cval=0,
    rescale=1. / 255,
    #rotation_range=90,
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    #horizontal_flip=True,
    #vertical_flip=True,
    preprocessing_function=crop_image,
    validation_split=0.0)

test_image_dir = './images_test_rev1/'

test_generator = testdatagen.flow_from_dataframe(
    dataframe=testdf,
    directory=test_image_dir,
    x_col="id",
    y_col=classes,
    subset="training",
    batch_size=64,
    seed=123,
    shuffle=False,
    class_mode="raw",
    target_size=(64, 64))

In [ ]:
test_pred = model.predict(test_generator, verbose=1)

In [ ]:
testdf[classes] = test_pred
testdf.drop(['id'])
testdf.to_csv(data_dir+'test_submission.csv', index=False)